In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
file_path = 'C:\\Users\\Dell\\Downloads\\Garstec_AS09_chiara.hdf5'

In [3]:
# Inputs
ages = []
massini = []
fehini = []
alphamlt = []
yini = []
eta = []
alphafe = []

# Outputs
teff = []
luminosity = []
dnufit = []
FeH = []
G_GAIA = []
massfin = []
numax = []
MeH = []


# Open the hdf5 file (read-only mode)
with h5py.File(file_path, 'r') as hdf:

    grid = hdf['grid']
    tracks = grid['tracks']

    # Get a list of track names and shuffle for random sampling
    track_names = list(tracks.keys())
    random.seed(1)
    random.shuffle(track_names)

    # Choose a subset of tracks to process
    num_tracks = 1000  # Set the number of tracks to process
    selected_tracks = track_names[:num_tracks]

    for track_name in selected_tracks:  # Iterate over the selected track names
        track = tracks[track_name]
        
        # Inputs
        epsilon = 1e-10  # Small constant to handle zero values
        age_data = np.sign(track['age'][:]) * np.log(np.abs(track['age'][:]) + epsilon)
        mass_data = np.sign(track['massini'][:]) * np.log(np.abs(track['massini'][:]) + epsilon)
        FeHini_data = np.sign(track['FeHini'][:]) * np.log(np.abs(track['FeHini'][:]) + epsilon)
        alphaMLT_data = np.sign(track['alphaMLT'][:]) * np.log(np.abs(track['alphaMLT'][:]) + epsilon)
        yini_data = np.sign(track['yini'][:]) * np.log(np.abs(track['yini'][:]) + epsilon)
        eta_data = np.sign(track['eta'][:]) * np.log(np.abs(track['eta'][:]) + epsilon)
        alphaFe_data = np.sign(track['alphaFe'][:]) * np.log(np.abs(track['alphaFe'][:]) + epsilon)

        ages.append(age_data)
        massini.append(mass_data)
        fehini.append(FeHini_data)
        alphamlt.append(alphaMLT_data)
        yini.append(yini_data)
        eta.append(eta_data)
        alphafe.append(alphaFe_data)

        # Outputs
        teff.append(track['Teff'][:])
        luminosity.append(track['LPhot'][:])
        dnufit.append(track['dnufit'][:])
        FeH.append(track['FeH'][:])
        G_GAIA.append(track['G_GAIA'][:])
        massfin.append(track['massfin'][:])
        numax.append(track['numax'][:])
        MeH.append(track['MeH'][:])

# Convert lists to numpy arrays and concatenate them (make one big list)

# Inputs
ages = np.concatenate(ages).reshape(-1, 1)
massini = np.concatenate(massini).reshape(-1, 1)
fehini = np.concatenate(fehini).reshape(-1, 1)
alphamlt = np.concatenate(alphamlt).reshape(-1, 1)
yini = np.concatenate(yini).reshape(-1, 1)
eta = np.concatenate(eta).reshape(-1, 1)
alphafe = np.concatenate(alphafe).reshape(-1, 1)

# Outputs
teff = np.concatenate(teff)
luminosity = np.concatenate(luminosity)
dnufit = np.concatenate(dnufit).reshape(-1, 1)
FeH = np.concatenate(FeH).reshape(-1, 1)
G_GAIA = np.concatenate(G_GAIA).reshape(-1, 1)
massfin = np.concatenate(massfin).reshape(-1, 1)
numax = np.concatenate(numax).reshape(-1, 1)
MeH = np.concatenate(MeH).reshape(-1, 1)

# Take the log of Teff and LPhot
log_teff = np.log10(teff).reshape(-1, 1)
log_luminosity = np.log10(luminosity).reshape(-1, 1)

# Combine all inputs into a single array
inputs = np.column_stack((ages, massini, fehini, alphamlt, yini, eta, alphafe))

# Combine Teff and LPhot as outputs
outputs = np.column_stack((log_teff, log_luminosity, dnufit, FeH, massfin, numax, MeH))

In [4]:
X_scaler = StandardScaler()
Y_scaler = StandardScaler()

X_train, X_test, Y_train, Y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=1)

X_train = torch.FloatTensor(X_scaler.fit_transform(X_train))
X_test = torch.FloatTensor(X_scaler.transform(X_test))
Y_train = torch.FloatTensor(Y_scaler.fit_transform(Y_train))
Y_test = torch.FloatTensor(Y_scaler.transform(Y_test))

dataset = TensorDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  

In [5]:
# Define the neural network model
class StellarModel(nn.Module):
    def __init__(self):
        super(StellarModel, self).__init__()
        self.fc1 = nn.Linear(7, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 7)  

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [6]:
# Initiate the model, define loss and optimizer
model = StellarModel()
criterion = nn.MSELoss()
lr = 0.005
step_size = 500
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.9, last_epoch=-1)

num_epochs = 600
train_loss = []  
test_loss = []   
learning_rate = []

In [7]:
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    # Forward pass for training data
    outputs_train = model(X_train)
    epoch_train_loss = criterion(outputs_train, Y_train)

    # Backward pass and optimization for training data
    optimizer.zero_grad()  # Clear gradients
    epoch_train_loss.backward()  # Backpropagation
    optimizer.step()       # Update weights
    

    # Forward pass for test data
    model.eval()  # Set the model to evaluation mode (disables dropout, etc.)
    with torch.no_grad():
        outputs_test = model(X_test)
        epoch_test_loss = criterion(outputs_test, Y_test)  # Calculate test loss
    
    scheduler.step()
    lr_after  = optimizer.param_groups[0]["lr"]
    # Store the losses for plotting
    train_loss.append(epoch_train_loss.item())
    test_loss.append(epoch_test_loss.item())
    learning_rate.append(lr_after)

    # Print progress every 1000 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {epoch_train_loss.item():.4f}, Test Loss: {epoch_test_loss.item():.4f}')

# Plot training and test loss
plt.subplot(1, 2, 1)
plt.plot(np.log10(train_loss), label='Training Loss (log scale)')
plt.plot(np.log10(test_loss), label='Test Loss (log scale)')
plt.xlabel('Epoch')
plt.ylabel('Log Loss')
plt.title('Log Loss vs. Epoch (Training vs. Test)')

plt.subplot(1, 2, 2)
plt.plot(learning_rate)
plt.xlabel('Epoch')
plt.ylabel('Learning rate')
plt.title('Learning rate vs. Epoch')
plt.legend()
plt.show()

Epoch [100/600], Train Loss: 0.4593, Test Loss: 0.4570
Epoch [200/600], Train Loss: 0.4716, Test Loss: 0.4720
Epoch [300/600], Train Loss: 0.4587, Test Loss: 0.4567
Epoch [400/600], Train Loss: 0.4727, Test Loss: 0.4702


KeyboardInterrupt: 

In [ ]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'garstec_model_stateV2.pth')

# Save the optimizer's state dictionary
torch.save(optimizer.state_dict(), 'garstec_optimizer_stateV2.pth')

# Save the entire model
torch.save(model, 'garstec_entire_modelV2.pth')

print("Model, optimizer, and entire model saved successfully.")